In [0]:
pwd

'/content'

In [0]:
from google.colab import drive
drive.mount('./drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at ./drive


In [0]:
!ls './drive/My Drive/Thesis Work'

 CambridgeMonoPhoneDf.pickle	      Implementation3.ipynb
 CambridgeWordFeaturesDf.pickle       Implementation4
 CheckScrapSpeed.ipynb		      Implementation4.ipynb
 ConvertToWav.ipynb		      L2monoPhoneDfTest.pickle
'Copy of l2arctic_release_v2.0.zip'   L2monoPhoneDfTrain.pickle
 DelIfCopiesFine.ipynb		      L2WordFeaturesDfTest.pickle
 Implementation1		      L2WordFeaturesDfTrain.pickle
 Implementation1.ipynb		      MostCommon3000Oxford.txt
 Implementation2		      UKpronunciations
 Implementation2.ipynb		      USpronunciations
 Implementation3


Bism  
# Imports

In [0]:
import pandas as pd
import numpy as np
import os
import IPython.display as ipd
import librosa
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
matplotlib.style.use('ggplot')
from tqdm import tqdm_notebook


#SKLEARN
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
import sklearn.metrics as sklm
from sklearn.manifold import TSNE

#KERAS
from keras import models
from keras.models import Model, load_model
from keras import callbacks
import keras
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose, AveragePooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import SGD, Adam

Using TensorFlow backend.


# Implementation 4  (PHONE LEVEL)
## Using L2Training set
## L2 as val+test set

### Training

#### MODEL

In [0]:
x_trainALL = np.load('./drive/My Drive/Thesis Work/Implementation4/x_train.npy')
train_phones = np.load('./drive/My Drive/Thesis Work/Implementation4/train_phones.npy')

In [0]:
train_phones.shape

(65113,)

In [0]:
x_trainALL.shape

(65113, 13, 50, 3)

In [0]:
#commented to avoid accedental runs
'''for phone in set(train_phones):
  np.save("./drive/My Drive/Thesis Work/Implementation4/phone_means/mean_"+phone+".npy",\
          np.max(np.abs(x_train[train_phones == phone]),(0,1,2)))'''

In [0]:
x_train = np.array([]).reshape(0,13,50,3)
for phone in list(set(train_phones)):
  train_max = np.load('./drive/My Drive/Thesis Work/Implementation4/phone_means/mean_'+phone+".npy")
  x_train = np.vstack((x_train,x_trainALL[train_phones == phone] / train_max))

In [0]:
x_train.shape

(65113, 13, 50, 3)

In [0]:
print(np.max(x_train))
print(np.min(x_train))

0.9818808268950413
-1.0


In [0]:
np.nan in x_train

False

In [0]:
#together then alehda

In [0]:
input_img = Input(shape=(13, 50, 3))  # adapt this if using `channels_first` image data format

x = Conv2D(8, (2, 3), activation='tanh', padding='valid')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(4, (3, 3), activation='tanh', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (3, 12, 32)

x = BatchNormalization()(encoded)
x = Conv2D(4, (3, 3), activation='tanh', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = BatchNormalization()(x)
x = Conv2D(8, (3, 3), activation='tanh', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = BatchNormalization()(x)
decoded = Conv2DTranspose(3, (2, 3), activation='tanh', padding='valid')(x)

autoencoder = Model(input_img, decoded)

autoencoder.compile(optimizer= keras.optimizers.Adam(), loss='mean_squared_error');

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 13, 50, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 12, 48, 8)         152       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 24, 8)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 6, 24, 8)          32        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 6, 24, 4)          292       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 12, 4)          0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 3, 12, 4)          16        
__________

In [0]:
callbacklist=[callbacks.History(),callbacks.ModelCheckpoint('./jawadmodel.h5', monitor='loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=5)]
autoencoder.fit(x_train, x_train, batch_size=8192, epochs=500, callbacks=callbacklist)

Instructions for updating:
Use tf.cast instead.
Epoch 1/500
65113/65113 [==============================] - 13s 203us/step - loss: 0.4426
Epoch 2/500
65113/65113 [==============================] - 9s 136us/step - loss: 0.3388
Epoch 3/500
65113/65113 [==============================] - 9s 138us/step - loss: 0.2682
Epoch 4/500
65113/65113 [==============================] - 9s 139us/step - loss: 0.2119
Epoch 5/500
65113/65113 [==============================] - 9s 139us/step - loss: 0.1634

Epoch 00005: loss improved from inf to 0.16335, saving model to ./jawadmodel.h5
Epoch 6/500
65113/65113 [==============================] - 9s 138us/step - loss: 0.1209
Epoch 7/500
65113/65113 [==============================] - 9s 135us/step - loss: 0.0863
Epoch 8/500
65113/65113 [==============================] - 9s 138us/step - loss: 0.0585
Epoch 9/500
65113/65113 [==============================] - 9s 139us/step - loss: 0.0398
Epoch 10/500
65113/65113 [==============================] - 9s 139us/step - lo

In [0]:
!cp ./jawadmodel.h5 "./drive/My Drive/Thesis Work/Implementation4/autoencoderModelALL.h5"

### Testing

In [0]:
!ls './drive/My Drive/Thesis Work/Implementation4'

autoencoderModelALL.h5	test_labels.npy       train_phones.npy	y_train.npy
model.h5		test_phones.npy       x_test.npy
phone_means		test_typeOfError.npy  x_train.npy


In [0]:
x_testALL = np.load('./drive/My Drive/Thesis Work/Implementation4/x_test.npy')
test_phones = np.load('./drive/My Drive/Thesis Work/Implementation4/test_phones.npy')
test_labelsALL = np.load('./drive/My Drive/Thesis Work/Implementation4/test_labels.npy')

In [0]:
#MAX normalization
x_test = np.array([]).reshape(0,13,50,3)
test_labels = np.array([]).reshape(0,1)
for phone in set(test_phones).intersection(set(train_phones)):
  train_max = np.load('./drive/My Drive/Thesis Work/Implementation4/phone_means/mean_'+phone+".npy")
  x_test = np.vstack((x_test,x_testALL[test_phones == phone] / train_max))
  test_labels = np.vstack((test_labels, test_labelsALL[test_phones == phone].reshape(-1,1)))

In [0]:
test_labels = test_labels.reshape(-1)

In [0]:
x_test.shape

(34795, 13, 50, 3)

In [0]:
shuffledIndexes = np.random.permutation(range(x_test.shape[0]))
x_test = x_test[shuffledIndexes]
test_labels = test_labels[shuffledIndexes]

In [0]:
validationSetLength = 6000 #initial 6000 from test as validation set
accs = []
f1mis = []

for threshold in [0.0010,0.00105,0.00120,0.00125,0.00130]:
  predictions = [1 if autoencoder.evaluate(x_test[i:i+1],x_test[i:i+1],verbose=0)<=threshold\
   else 0 for i in range(validationSetLength)]
  
  accs.append(sklm.accuracy_score(test_labels[:validationSetLength],predictions))
  f1mis.append(sklm.precision_recall_fscore_support(test_labels[:validationSetLength],predictions)[2][0])

print("Threshold: ", np.argmax(f1mis))
print("Max Achievable Accuracy: ", accs[np.argmax(f1mis)])
print("Max Achievable F-1 score for Mispronunciations: ", max(f1mis))


Threshold:  1
Max Achievable Accuracy:  0.4558333333333333
Max Achievable F-1 score for Mispronunciations:  0.20886842742912526


In [0]:
validationSetLength = 6000 #initial 6000 from test as validation set
threshold = 0.00105

predictions = [1 if autoencoder.evaluate(x_test[i:i+1],x_test[i:i+1],verbose=0)<=threshold\
 else 0 for i in range(validationSetLength,x_test.shape[0])] 

In [0]:
print('Accuracy = ', sklm.accuracy_score(test_labels[validationSetLength:],predictions))

Accuracy =  0.4498003125542629


In [0]:
sklm.precision_recall_fscore_support(test_labels[validationSetLength:],predictions) #0 - 1

(array([0.12207055, 0.89312852]),
 array([0.60708922, 0.42923899]),
 array([0.2032688, 0.579817 ]),
 array([ 3329, 25466]))

If a voice is substituted by another then combined autoencoder will reconstruct it also as trained on that also so combined is not working AH